# 프로젝트 - Movielens 영화 추천 실습

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

In [1]:
# !mkdir -p ~/aiffel/Recommendation_System/2_recommender_process/data/ml-1m
# !ln -s ~/data/ml-1m/* ~/aiffel/Recommendation_System/2_recommender_process/data/ml-1m

In [2]:
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


## 1) 데이터 준비와 전처리

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [3]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/Recommendation_System/2_recommender_process/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [6]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [7]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'counts']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [8]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/Recommendation_System/2_recommender_process/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

## 2) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [9]:
merged_df = pd.merge(ratings, movies, on='movie_id')
merged_df

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [10]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'title', 'counts']
merged_df = merged_df[using_cols]
merged_df

,user_id,title,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5
...,...,...,...
836473,5851,One Little Indian (1973),5
836474,5854,Slaughterhouse (1987),4
836475,5854,"Promise, The (Versprechen, Das) (1994)",3
836476,5938,"Five Wives, Three Secretaries and Me (1998)",4


In [11]:
# title 컬럼의 이름을 movie로 바꿉니다.
merged_df.rename(columns={'title':'movie'}, inplace=True)

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [12]:
# 유저 수
merged_df['user_id'].nunique()

6039

In [13]:
# 영화 수
merged_df['movie'].nunique()

3628

In [14]:
# 인기 많은 영화
movie_count = merged_df.groupby('movie')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [15]:
# 유저별 몇 개의 영화를 봤는지에 대한 통계
user_count = merged_df.groupby('user_id')['movie'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie, dtype: float64

## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [16]:
merged_df['user_id'].max()

6040

In [17]:
merged_df.isin({'user_id': [6210]})['user_id'].any()

False

In [18]:
merged_df

,user_id,movie,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5
...,...,...,...
836473,5851,One Little Indian (1973),5
836474,5854,Slaughterhouse (1987),4
836475,5854,"Promise, The (Versprechen, Das) (1994)",3
836476,5938,"Five Wives, Three Secretaries and Me (1998)",4


In [19]:
# 목록에 있는 좋아하는 영화
my_favorite = ['Shawshank Redemption, The (1994)' , 'Saving Private Ryan (1998)' ,
               "Schindler's List (1993)" ,'Psycho (1960)' ,'Scarlet Letter, The (1995)']

my_playlist = pd.DataFrame({'user_id': [6210]*5, 'movie': my_favorite, 'counts': [5]*5})

if not ratings.isin({'user_id': [6210] })['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    merged_df = pd.concat([merged_df, my_playlist])                            # 위에 임의로 만든 my_playlist 데이터를 추가해 줍니다. 

merged_df[merged_df['user_id']==6210]       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie,counts
0,6210,"Shawshank Redemption, The (1994)",5
1,6210,Saving Private Ryan (1998),5
2,6210,Schindler's List (1993),5
3,6210,Psycho (1960),5
4,6210,"Scarlet Letter, The (1995)",5


In [20]:
merged_df.tail(6)

,user_id,movie,counts
836477,5948,Identification of a Woman (Identificazione di ...,5
0,6210,"Shawshank Redemption, The (1994)",5
1,6210,Saving Private Ryan (1998),5
2,6210,Schindler's List (1993),5
3,6210,Psycho (1960),5
4,6210,"Scarlet Letter, The (1995)",5


In [21]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = merged_df['user_id'].unique()
movie_unique = merged_df['movie'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [22]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[6210])
print(movie_to_idx['Saving Private Ryan (1998)'])

6039
48


In [23]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = merged_df['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(merged_df):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    merged_df['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = merged_df['movie'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(merged_df):
    print('movie column indexing OK!!')
    merged_df['movie'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

merged_df

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie,counts
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,157,5
1,6039,48,5
2,6039,23,5
3,6039,1028,5


## 4) CSR matrix를 직접 만들어 봅시다.

In [24]:
from scipy.sparse import csr_matrix

num_user = merged_df['user_id'].nunique()
num_movie = merged_df['movie'].nunique()

csr_data = csr_matrix((merged_df.counts, (merged_df.user_id, merged_df.movie)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

#### ㄴ ( 유저 수 * 영화 수 ) 크기의 rating matrix 생성

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [25]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [26]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=100, dtype=np.float32)

In [27]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [28]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/100 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [29]:
Me, Ryan = user_to_idx[6210], movie_to_idx['Saving Private Ryan (1998)']
My_vector, Ryan_vector = als_model.user_factors[Me], als_model.item_factors[Ryan]

In [30]:
My_vector

array([-0.01736044, -0.02453215, -0.4377755 ,  0.1056853 ,  0.39120364,
       -0.26352823, -0.15416017,  0.45508343, -0.2590379 , -0.6399636 ,
        0.03669723, -0.05252836, -0.14825116, -0.18333948,  0.4754534 ,
       -0.13354337,  0.06597464,  0.01697549, -0.57361007, -0.49338743,
        0.4711071 ,  0.02536586,  0.23719501,  0.22791465,  0.18971004,
       -0.51568776,  0.1410552 , -0.40574077,  0.10748325, -0.00640422,
       -0.01266608,  0.21674286,  0.36524895, -0.04118828,  0.25299078,
        0.26781446,  0.07933213,  0.5280063 , -0.20302917, -0.18545401,
        0.11961699, -0.30504313,  0.09271952,  0.18687442,  0.13560866,
        0.30227965, -0.34219098, -0.0585837 ,  0.21812128,  0.02471867,
        0.06219133,  0.52243924, -0.00686734,  0.20043182,  0.3078249 ,
        0.19163597, -0.09487867,  0.17058358,  0.25000393, -0.10970465,
        0.08067943, -0.09088218, -0.48156616, -0.27723134, -0.25568154,
        0.10586345,  0.15525238, -0.2450547 , -0.04386389, -0.01

In [31]:
Ryan_vector

array([-4.38524876e-04,  5.96098490e-02, -6.09527975e-02,  1.39876725e-02,
        3.27461846e-02, -2.78714299e-02, -1.80334337e-02,  8.34664777e-02,
       -2.31895056e-02, -8.42476338e-02,  7.86486343e-02, -1.78693179e-02,
       -3.13214287e-02, -7.19648823e-02,  1.62764452e-02, -2.01775488e-02,
        1.58806182e-02, -2.89711095e-02, -6.22656681e-02, -8.06460530e-02,
        4.52693142e-02,  6.39439048e-03,  8.36121012e-03, -3.36358324e-02,
        6.30651638e-02, -4.18374836e-02,  3.06452662e-02, -3.37465405e-02,
       -1.97929144e-02, -2.14257911e-02, -2.15999391e-02, -1.05391315e-03,
        4.37632017e-02,  6.02765642e-02,  3.72611061e-02,  3.50774378e-02,
        3.51743365e-04,  7.46036321e-02, -2.09564511e-02, -1.42020751e-02,
       -8.78029596e-03, -5.50339706e-02,  5.36483452e-02,  1.30857062e-03,
        4.56103720e-02, -4.16108267e-03, -3.98270600e-02, -6.24809563e-02,
        1.10631054e-02,  1.07169664e-02,  5.07560112e-02,  2.90373527e-02,
       -7.01180473e-03,  

In [32]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(My_vector, Ryan_vector)

0.7228996

#### ㄴ 모델이 예측한 나의 '라이언 일병 구하기' 선호도
: '라이언 일병 구하기'는 1에 가까운 유의미한 dot product 결과가 나왔다.

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 영화를 찾습니다.

In [35]:
favorite_movie = "Schindler's List (1993)"
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(23, 1.0),
 (157, 0.709825),
 (48, 0.63776946),
 (121, 0.54937226),
 (248, 0.4529315),
 (51, 0.39929307),
 (87, 0.39620173),
 (269, 0.36098868),
 (222, 0.35442835),
 (160, 0.35030138),
 (3517, 0.34223056),
 (323, 0.3354709),
 (450, 0.3108138),
 (3493, 0.30550745),
 (1370, 0.3041754)]

In [36]:
#movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

["Schindler's List (1993)",
 'Shawshank Redemption, The (1994)',
 'Saving Private Ryan (1998)',
 'Silence of the Lambs, The (1991)',
 'Good Will Hunting (1997)',
 'Fargo (1996)',
 'Braveheart (1995)',
 'GoodFellas (1990)',
 'Pulp Fiction (1994)',
 'Forrest Gump (1994)',
 'Paralyzing Fear: The Story of Polio in America, A (1998)',
 'Dead Man Walking (1995)',
 'Life Is Beautiful (La Vita è bella) (1997)',
 'Held Up (2000)',
 'Killing Fields, The (1984)']

몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인

In [56]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie_raw = als_model.similar_items(movie_id)
    similar_movie = [[idx_to_movie[i[0]],i[1]] for i in similar_movie_raw]
    return similar_movie

In [57]:
get_similar_movie('Shawshank Redemption, The (1994)')

[['Shawshank Redemption, The (1994)', 0.99999994],
 ['Silence of the Lambs, The (1991)', 0.7736896],
 ["Schindler's List (1993)", 0.709825],
 ['Good Will Hunting (1997)', 0.6697185],
 ['Pulp Fiction (1994)', 0.6688082],
 ['Fargo (1996)', 0.6245397],
 ['GoodFellas (1990)', 0.5744428],
 ['Saving Private Ryan (1998)', 0.47370625],
 ['Dead Man Walking (1995)', 0.43419254],
 ['Usual Suspects, The (1995)', 0.40799075]]

In [58]:
get_similar_movie("Psycho (1960)")

[['Psycho (1960)', 0.99999994],
 ['Rear Window (1954)', 0.5963216],
 ['Birds, The (1963)', 0.5493949],
 ['Shining, The (1980)', 0.54454446],
 ['Vertigo (1958)', 0.53614473],
 ['Exorcist, The (1973)', 0.4939159],
 ["Rosemary's Baby (1968)", 0.4909461],
 ['Poltergeist (1982)', 0.4751233],
 ['Omen, The (1976)', 0.4518824],
 ['Carrie (1976)', 0.43581325]]

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [41]:
user = user_to_idx[6210]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(121, 0.5371063),
 (87, 0.4316099),
 (248, 0.345795),
 (0, 0.2968178),
 (51, 0.29553878),
 (99, 0.2941103),
 (222, 0.29110637),
 (269, 0.263489),
 (120, 0.2283895),
 (47, 0.22425127),
 (993, 0.20514062),
 (487, 0.20124502),
 (323, 0.18899128),
 (1194, 0.18671532),
 (569, 0.18579304),
 (197, 0.18277457),
 (472, 0.17369115),
 (450, 0.17272112),
 (22, 0.17190285),
 (432, 0.1697753)]

In [59]:
[[idx_to_movie[i[0]],i[1]] for i in movie_recommended]

[['Silence of the Lambs, The (1991)', 0.5371063],
 ['Braveheart (1995)', 0.4316099],
 ['Good Will Hunting (1997)', 0.345795],
 ["One Flew Over the Cuckoo's Nest (1975)", 0.2968178],
 ['Fargo (1996)', 0.29553878],
 ['American Beauty (1999)', 0.2941103],
 ['Pulp Fiction (1994)', 0.29110637],
 ['GoodFellas (1990)', 0.263489],
 ['Raiders of the Lost Ark (1981)', 0.2283895],
 ['To Kill a Mockingbird (1962)', 0.22425127],
 ['Shining, The (1980)', 0.20514062],
 ['Boat, The (Das Boot) (1981)', 0.20124502],
 ['Dead Man Walking (1995)', 0.18899128],
 ['Fatal Attraction (1987)', 0.18671532],
 ['Rear Window (1954)', 0.18579304],
 ['Jaws (1975)', 0.18277457],
 ['Sling Blade (1996)', 0.17369115],
 ['Life Is Beautiful (La Vita è bella) (1997)', 0.17272112],
 ['Back to the Future (1985)', 0.17190285],
 ['Taxi Driver (1976)', 0.1697753]]

제 1순위 추천 영화 : 'Silence of the Lambs, The (1991)'

AlternatingLeastSquares 클래스에 구현된 explain 메서드를 사용하여 이 추천에 기여한 정도를 확인

In [60]:
Lambs = movie_to_idx['Silence of the Lambs, The (1991)']
explain = als_model.explain(user, csr_data, itemid=Lambs)

In [61]:
explain

(0.5343719711697028,
 [(157, 0.2847829554566777),
  (23, 0.15410308450130758),
  (1028, 0.07749186630170957),
  (48, 0.014531075605823231),
  (2908, 0.0034629893041847153)],
 (array([[ 0.99997503,  0.06497513,  0.08020814, ...,  0.10912434,
           0.06937606,  0.01918197],
         [ 0.0649735 ,  1.04748107,  0.01556786, ...,  0.02886214,
           0.03183621,  0.02567027],
         [ 0.08020613,  0.02151858,  1.03872904, ...,  0.07111416,
           0.12133944,  0.01432407],
         ...,
         [ 0.10912161,  0.03732292,  0.08307033, ...,  0.96444247,
           0.01012547, -0.00234113],
         [ 0.06937433,  0.03785555,  0.13209895, ...,  0.06327574,
           0.95286104,  0.01269385],
         [ 0.01918149,  0.02813547,  0.01681701, ...,  0.0293388 ,
           0.07330167,  0.94522823]]),
  False))

In [62]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Shawshank Redemption, The (1994)', 0.2847829554566777),
 ("Schindler's List (1993)", 0.15410308450130758),
 ('Psycho (1960)', 0.07749186630170957),
 ('Saving Private Ryan (1998)', 0.014531075605823231),
 ('Scarlet Letter, The (1995)', 0.0034629893041847153)]

In [63]:
get_similar_movie('Silence of the Lambs, The (1991)')

[['Silence of the Lambs, The (1991)', 1.0000001],
 ['Shawshank Redemption, The (1994)', 0.7736896],
 ['Fargo (1996)', 0.75091213],
 ['Pulp Fiction (1994)', 0.66356075],
 ["Schindler's List (1993)", 0.54937226],
 ['GoodFellas (1990)', 0.5053628],
 ['Usual Suspects, The (1995)', 0.47057202],
 ['Sixth Sense, The (1999)', 0.46859443],
 ['Good Will Hunting (1997)', 0.44624415],
 ['Sling Blade (1996)', 0.41415766]]

### MF모델이 예측한 1순위 추천 분석.
1. 쇼생크 탈출, 주홍글씨는 소설을 영화화한 것이다. 1순위 추천 영화인 양들의 침묵 또한 소설을 바탕으로 만든 영화이다.
2. 영들의 침묵에는 연쇄살인자를 추적하는 스릴러 영화이다. 영화 'Psycho' 또한 연쇄살인마가 나오는 공포 영화라는 점에서 유사성을 찾을 수 있다.

# 회고

- 배운 점
1. SVD의 원리와 기하학적 의미를 이해할 수 있었다. 고윳값 분해에서는 선형변환 후 크기와 상관없이 방향이 일정한 고유벡터와 그 고윳값을 통하여 행렬의 정보량을 나타내는 벡터를 찾았다. SVD에서는 선형변환 후 직교하는 특이벡터와 그 특이값을 통해 행렬의 정보량을 파악할 수 있었다. 이 때 직교하는 벡터를 찾은 이유는 Orthogonal Matrix의 성질을 활용할 수 있기 때문임을 이해할 수 있었다.
2. Matrix Factorization은 데이터로 주어진 rating matrix에 근거하여 latent feature를 gradient decendant 방법에 따라 최적화하여, user matrix와 item matrix의 matrix multiplication을 통해 rating matrix에서 얻지 못한 element의 값을 추정할 수 있는 방법으로 이해하였다. 이번 노드를 공부하면서 이전에 학습하였던 LSA, LDA에 대한 원리 또한 이해하는데 도움이 되었다.

- 문제해결을 시도한 점
1. CSR matrix를 생성하는 과정에서 ValueError: row index exceeds matrix dimensions가 발생하였다. 원인이 제대로 이해가 되지 않았지만, rating matrix에 속한 user_id와 movie의 인덱스를 최신화하는 과정에서 오류가 발생하지 않게 되었다.

- 추가적으로 시도할만 한 것들
1. MF 방법은 주어진 데이터에 따라 feature를 얻어내는 self-supervised learning이라는 것을 알 수 있었다. 즉, 사용자의 행동 데이터에 의해 결정된 feature를 rating matrix에 존재하는 데이터와의 Error를 최소화하는 범위에서 최적화하는 것이다. 이로 인해 local minimun에 빠지는 문제가 발생할 수 있는데, '주홍글씨'의 선호도가 낮은 결과가 이에 해당하는 것으로 이해하였다. 
2. 이는 협업 필터링 방법에서 발생할 수 있는 문제로 확인하였고, 추후 item 정보를 바탕으로 유사도를 측정하는 homogeneous method을 통해 협업 필터링에서 발생하는 문제를 해결할 아이디어를 이해하고자 한다.